# Portfolio Optimization - Part 2

**Note:** This notebook depends on variables from `part1_Matrix_algebra_julia.ipynb`. 
Please run Part 1 first to define:
- `ret_df` - DataFrame with log returns
- `ret_matrix` - Matrix version of returns
- `numeric_cols` - Column names (ticker symbols)
- `W` - Equal weights vector
- `cov_matrix` - Covariance matrix

Alternatively, you can run the data loading cells from Part 1 before running this notebook.

In [ ]:
using Optim
using LinearAlgebra
using Statistics
using PlotlyJS

# Optimization - This is the function we're minimizing
function sharpe_pf(W, returns_matrix)
    cov_mat = cov(returns_matrix, dims=1)
    mu = vec(mean(returns_matrix, dims=1))
    pf_risk = sqrt(W' * cov_mat * W)
    SR = dot(W, mu) / pf_risk
    return -SR
end

# Test with equal weights
sharpe_pf(W, ret_matrix)

In [ ]:
# === Precompute ===
mu = vec(mean(ret_matrix, dims=1))
cov_mat = cov(ret_matrix, dims=1)
n_assets = size(ret_matrix, 2)

# === Objective Function ===
function neg_sharpe_lbfgsb(W)
    W_norm = W / sum(W)  # normalize to sum to 1
    risk = sqrt(W_norm' * cov_mat * W_norm)
    if risk == 0
        return Inf
    end
    return -(dot(W_norm, mu) / risk)
end

# === Initial Setup ===
initial_weights = ones(n_assets) / n_assets
lower_bounds = zeros(n_assets)
upper_bounds = ones(n_assets)

# === Run Optimizer ===
result = optimize(neg_sharpe_lbfgsb, lower_bounds, upper_bounds, initial_weights, Fminbox(LBFGS()),
    Optim.Options(show_trace=true, iterations=1000, g_tol=1e-8))

# === Results ===
if Optim.converged(result)
    weights_opt = result.minimizer / sum(result.minimizer)
    max_sharpe = -neg_sharpe_lbfgsb(weights_opt)
    
    println("\n✅ Optimization successful!")
    println("Optimization terminated successfully")
    println("Current function value: ", result.minimum)
    println("Iterations: ", result.iterations)
    println("\nMax Sharpe Ratio: ", max_sharpe)
    println("Optimal Weights:\n")
    for (ticker, weight) in zip(numeric_cols, weights_opt)
        if weight > 0
            println("$ticker: $(round(weight, digits=4))")
        end
    end
else
    println("\n❌ Optimization failed: ", result)
end

In [ ]:
# Precompute for speed
mu = vec(mean(ret_matrix, dims=1))
cov_mat = cov(ret_matrix, dims=1)
n_assets = size(ret_matrix, 2)

# Objective function with normalization inside
function neg_sharpe_lbfgsb(W)
    W_norm = W / sum(W)  # normalize weights to sum to 1
    risk = sqrt(W_norm' * cov_mat * W_norm)
    if risk == 0
        return Inf
    end
    return -(dot(W_norm, mu) / risk)
end

# Initial guess
initial_weights = ones(n_assets) / n_assets

# Bounds (no short selling)
lower_bounds = zeros(n_assets)
upper_bounds = ones(n_assets)

# Run optimizer
result = optimize(neg_sharpe_lbfgsb, lower_bounds, upper_bounds, initial_weights, Fminbox(LBFGS()),
    Optim.Options(iterations=300))

# Normalize final weights
weights_opt = result.minimizer / sum(result.minimizer)

# Output
println("\n⚡️ FAST Optimization Results:")
println("Optimization terminated: ", Optim.converged(result))
println("Current function value: ", result.minimum)
println("Iterations: ", result.iterations)
println("\nMax Sharpe Ratio: ", -neg_sharpe_lbfgsb(weights_opt))
println("Optimal Weights:")
for (ticker, weight) in zip(numeric_cols, weights_opt)
    if weight > 0
        println("$ticker: $(round(weight, digits=4))")
    end
end

In [ ]:
# === STEP 1: Sharpe ratio function (negative, because we minimize) ===
function neg_sharpe_ratio(weights)
    portfolio_returns = ret_matrix * weights  # (T,)
    mean_return = mean(portfolio_returns)
    std_return = std(portfolio_returns)
    if std_return == 0
        return Inf  # avoid division by zero
    end
    return -mean_return / std_return
end

# === STEP 2: Constraints and bounds ===
n_assets = size(ret_matrix, 2)

# Initial guess: equal weighting
initial_weights = ones(n_assets) / n_assets

# Bounds: no short-selling (0 ≤ w ≤ 1)
lower_bounds = zeros(n_assets)
upper_bounds = ones(n_assets)

# === STEP 3: Optimization with constraint ===
# We use a constrained optimizer
# Define constraint: sum(w) = 1
function constraint_sum(w)
    return [sum(w) - 1.0]
end

# Use IPNewton for constrained optimization
res = optimize(neg_sharpe_ratio, 
    initial_weights,
    LBFGS(),
    Optim.Options(show_trace=true, iterations=1000))

# === STEP 4: Results ===
if Optim.converged(res)
    println("\n✅ Optimization successful!")
    println("Max Sharpe Ratio: ", -res.minimum)
    println("Optimal Weights:\n")
    for (ticker, weight) in zip(numeric_cols, res.minimizer)
        println("$ticker: $(round(weight, digits=4))")
    end
else
    println("\n❌ Optimization failed")
end

In [ ]:
for (ticker, weight) in zip(numeric_cols, res.minimizer)
    if weight > 0.001
        println("$ticker: $(round(weight, digits=4))")
    end
end

In [ ]:
res

In [ ]:
opt_W = res.minimizer

In [ ]:
opt_W

In [ ]:
# Return for optimal weight
cumsum(ret_matrix * opt_W)

In [ ]:
# Return for equal weight
cumsum(ret_matrix * W)

In [ ]:

# === Step 1: Compute cumulative portfolio return (linear, not log) ===
portfolio_returns = ret_matrix * opt_W
cumulative_return = cumsum(portfolio_returns)  # No exp!

# === Step 2: Create interactive plot ===
trace = scatter(
    x=ret_df.Date,
    y=cumulative_return,
    mode="lines",
    name="Optimized Portfolio",
    line=attr(width=2, color="green")
)

# === Step 3: Customize layout ===
layout = Layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    width=1000,
    height=600,
    template="plotly_white"
)

plot(trace, layout)

In [ ]:

# === Step 1: Compute cumulative portfolio return (linear, not log) ===
portfolio_returns = ret_matrix * opt_W
cumulative_return = cumsum(portfolio_returns)  # No exp!

# === Step 2: Create interactive plot ===
trace = scatter(
    x=ret_df.Date,
    y=cumulative_return,
    mode="lines",
    name="Optimized Portfolio",
    line=attr(width=2, color="green")
)

# === Step 3: Customize layout ===
layout = Layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    width=1000,
    height=600,
    template="plotly_white"
)

plot(trace, layout)

In [ ]:

# === Step 1: Compute cumulative portfolio return ===
portfolio_returns = ret_matrix * opt_W
cumulative_return = exp.(cumsum(portfolio_returns))  # $1 growth over time

# === Step 2: Create interactive plot ===
trace = scatter(
    x=ret_df.Date,
    y=cumulative_return,
    mode="lines",
    name="Optimized Portfolio",
    line=attr(width=2, color="green")
)

# === Step 3: Customize layout ===
layout = Layout(
    title="📈 Interactive Cumulative Return of Optimized Portfolio",
    xaxis_title="Date",
    yaxis_title="Portfolio Value (\$1 Start)",
    width=1000,
    height=600,
    template="plotly_white"
)

plot(trace, layout)

In [ ]:
size(ret_matrix, 1) * 0.6

In [ ]:
train_idx = 1:Int(floor(size(ret_matrix, 1) * 0.6))
train = ret_matrix[train_idx, :]

In [ ]:
train

In [ ]:
test_idx = (Int(floor(size(ret_matrix, 1) * 0.6)) + 1):size(ret_matrix, 1)
test = ret_matrix[test_idx, :]

In [ ]:
test

In [ ]:
using Optim

# === Step 0: Select Top 550 Most Volatile Assets ===
asset_volatility = vec(std(train, dims=1))
top_550_indices = sortperm(asset_volatility, rev=true)[1:min(550, length(asset_volatility))]
train_reduced = train[:, top_550_indices]
top_550_tickers = numeric_cols[top_550_indices]

# === Step 1: Precompute everything ===
mu = vec(mean(train_reduced, dims=1))
cov_mat = cov(train_reduced, dims=1)
n_assets = length(mu)

# === Step 2: Define fast Sharpe function ===
function neg_sharpe(weights)
    weights_norm = weights / sum(weights)  # Normalize
    port_return = dot(weights_norm, mu)
    port_std = sqrt(weights_norm' * cov_mat * weights_norm)
    return port_std == 0 ? Inf : -port_return / port_std
end

# === Step 3: Optimization Setup ===
W0 = ones(n_assets) / n_assets
lower_bounds = zeros(n_assets)
upper_bounds = ones(n_assets)

# === Step 4: Run optimizer ===
res_train = optimize(neg_sharpe, lower_bounds, upper_bounds, W0, Fminbox(LBFGS()),
    Optim.Options(show_trace=true, iterations=1000))

# === Step 5: Results ===
if Optim.converged(res_train)
    weights_opt = res_train.minimizer / sum(res_train.minimizer)
    max_sharpe = -neg_sharpe(weights_opt)
    
    println("\n✅ Optimization Successful!")
    println("Max Sharpe Ratio (Train): $(round(max_sharpe, digits=6))")
    println("Iterations: ", res_train.iterations)
    println("\nTop Portfolio Weights (non-zero):\n")
    for (ticker, weight) in zip(top_550_tickers, weights_opt)
        if weight > 1e-4
            println("$ticker: $(round(weight, digits=4))")
        end
    end
else
    println("\n❌ Optimization Failed")
end

In [ ]:
res_train

In [ ]:
# Ensure test data uses the same assets as train_reduced
test_reduced = test[:, top_550_indices]

# Now it will work
cumulative_return = cumsum(test_reduced * res_train.minimizer)

# Optional: convert log returns to price
portfolio_value = exp.(cumulative_return)

portfolio_value

In [ ]:
cumsum(test * W)

In [ ]:
# Plot
using Plots

plot(ret_df.Date[test_idx], portfolio_value, 
    linewidth=2,
    title="Optimized Portfolio Cumulative Return on Test Data",
    xlabel="Date",
    ylabel="Portfolio Value (\$1 Start)",
    legend=false,
    size=(1200, 600),
    grid=true
)

In [ ]:

# === Step 1: Compute cumulative portfolio value from log returns ===
portfolio_log_returns = test_reduced * res_train.minimizer
portfolio_value = exp.(cumsum(portfolio_log_returns))  # Convert log return to price

# === Step 2: Create interactive plot ===
trace = scatter(
    x=ret_df.Date[test_idx],
    y=portfolio_value,
    mode="lines",
    name="Optimized Portfolio",
    line=attr(width=2, color="green")
)

# === Step 3: Customize layout ===
layout = Layout(
    title="Optimized Portfolio Cumulative Return on Test Data",
    xaxis_title="Date",
    yaxis_title="Portfolio Value (\$1 Start)",
    width=1000,
    height=600,
    template="plotly_white"
)

plot(trace, layout)

In [ ]:
sharpe_pf(res_train.minimizer, test_reduced)

In [ ]:
sharpe_pf(W, test)